In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai
import pymysql
import json
from datetime import datetime

In [ ]:
GEMINI_API_KEY = "INSERT_YOUR_API_KEY" # gemini api key
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="MySQL",
    database="dealership",
    autocommit=True,
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connection.cursor()

In [ ]:
PROBLEM_FILE = r"C:\YOUR_PATH\dealershipdb\docs\problem_statement.md"
VIEWS_OUTPUT_FILE = r"C:\YOUR_PATH\dealershipdb\docs\views_generated_by_ai.sql"

print("Loaded paths:")
print(PROBLEM_FILE)
print(VIEWS_OUTPUT_FILE)

In [ ]:
with open(PROBLEM_FILE, "r", encoding="utf-8") as f:
    problem_text = f.read()

print("Problem statement loaded:")
print(problem_text[:300], "...")

In [ ]:
model = genai.GenerativeModel("gemini-2.5-pro")

def generate_sql_views(problem_text: str) -> str:
    prompt = f"""
You are an AI SQL specialist. Based on the business problems below,
generate ONLY SQL VIEW definitions (CREATE VIEW ...) that solve them.
Do NOT generate explanations, only SQL.

Business Problems:
{problem_text}

Return ONLY SQL code.
"""
    response = model.generate_content(prompt)
    return response.text

In [ ]:
generated_views_sql = generate_sql_views(problem_text)

print("Generated SQL views:")
print(generated_views_sql)

In [ ]:
with open(VIEWS_OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(generated_views_sql)

print("Views saved to:", VIEWS_OUTPUT_FILE)

In [ ]:
statements = [
    stmt.strip()
    for stmt in generated_views_sql.split(";")
    if stmt.strip()
]

executed = 0

for stmt in statements:
    try:
        cursor.execute(stmt + ";")
        executed += 1
    except Exception as e:
        print("Error executing:")
        print(stmt)
        print("Error:", e)
        print()

print(f"Executed {executed} SQL view statements.")